In [2]:
import pandas as pd
import re
import string
import random
from transformers import RobertaTokenizer
from transformers import RobertaForSequenceClassification
import torch.nn as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AdamW 
import json
import torch
import numpy as np
import math
test = pd.read_csv('./sample_submission.csv')
model_path = './roberta/'

In [3]:
doc = pd.read_csv('./documents.csv')
doc_list = doc['doc_id'].tolist()
doc_text = doc['doc_text'].tolist()
doc_dict = {doc_list[i]: i for i in range(len(doc_list))}

In [4]:
train_q = pd.read_csv('./train_queries.csv')
train_query_data = [t.lower() for t in train_q['query_text']]
train_top_1000 = [t.split() for t in train_q['bm25_top1000']]
train_pos_list = [t.split() for t in train_q['pos_doc_ids']]
train_neg_list = [t[:] for t in train_top_1000] ##後800篇
train_pos_list = np.array(train_pos_list)
train_top_1000 = np.array(train_top_1000)
train_neg_list = np.array(train_neg_list) ##後970篇
print(train_neg_list.shape)

(120, 1000)


/home/nlp/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [5]:
## 裡面現在存的是pos doc idx
for pos_docs in range(train_pos_list.shape[0]):
    for idx in range(len(train_pos_list[pos_docs])):
        train_pos_list[pos_docs][idx] = doc_dict[train_pos_list[pos_docs][idx]]
        
for pos_docs in range(train_top_1000.shape[0]):
    for idx in range(len(train_top_1000[pos_docs])):
        train_top_1000[pos_docs][idx] = doc_dict[train_top_1000[pos_docs][idx]]
        
for pos_docs in range(train_neg_list.shape[0]):
    for idx in range(len(train_neg_list[pos_docs])):
        train_neg_list[pos_docs][idx] = doc_dict[train_neg_list[pos_docs][idx]]


In [6]:
def clean_string(content):
    content = content.replace('\n','.').replace('\t',',')# erease white space cause English name error
    content = re.sub("[+\.\/_,$%●▼►^*(+\"\']+|[+——~@#￥%……&*（）★]", "",content)
    return content.lower()

In [20]:
class TrainDataset(Dataset):
    def __init__(self, input_dict, y):
        self.input_ids = input_dict['input_ids']
#         self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']
        self.y = y
    def __getitem__(self, idx):
        input_id = self.input_ids[idx]
#         tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        y = self.y[idx]
        return input_id, attentionmask, y
    def __len__(self):
        return len(self.input_ids)

In [8]:
train_question = []
train_choice = []
train_y = []
N = 3 ## 錯的文章數量
for i in range(len(train_query_data)):
    for j in range(len(train_pos_list[i])):
        if type(doc_text[train_pos_list[i][j]]) != float:
            train_y.append(1)
            train_question.append(train_query_data[i])
            train_choice.append(clean_string(doc_text[train_pos_list[i][j]]))
            l = random.sample(range(0,999), N)
            count = -1
            while(count<N): ##有抽到前1000的
                if count != -1:
                    l = random.sample(range(0,999), N)
                count = 0 
                for check in l:
                    if check not in train_pos_list[i] and type(doc_text[int(train_neg_list[i][check])]) != float: ##不是POS
                        count += 1        
            for idx in l:
                train_y.append(0)
                train_question.append(train_query_data[i])
                train_choice.append(clean_string(doc_text[int(train_neg_list[i][idx])]))
# train_question = np.array(train_question)
# train_choice = np.array(train_choice)
# train_question = json.dumps(train_question)
train_y = np.array(train_y)

In [13]:
EVAL = 6000
test_q_set = train_question[:EVAL]
test_c_set = train_choice[:EVAL]
train_q_set = train_question[EVAL:]
train_c_set = train_choice[EVAL:]

test_y_set = train_y[:EVAL]
train_y_set = train_y[EVAL:]

In [12]:
## model 各參數
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
num_labels = 2
# tokenizer = RobertaTokenizer.from_pretrained(model_path)
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained(model_path,num_labels = num_labels)
model = model.to(device)
model.train()
EVAL = 6000
EPOCHS = 3
optimizer = AdamW(model.parameters(),lr = 3e-5)

Some weights of the model checkpoint at ./roberta/ were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ./roberta/ and are newly initialized: ['classifier.dense.weight', 'classifier.de

In [24]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

def get_test_acc(model, testloader):
    model.eval()  ##test mode
    total = 0 ##total_num
    correct = 0 ##correct_num
    with torch.no_grad():   ##eval不計算gradient 
        for data in testloader:
            tokens_tensors , masks_tensors,labels = [t.to(device) for t in data]
            outputs = model(input_ids = tokens_tensors,
                           attention_mask = masks_tensors,
                           labels = labels)
            pred = torch.argmax(outputs[1],dim=-1)
            total += labels.size()[0]
            correct += (pred == labels).sum().item()
    
    return correct/total

In [25]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
def get_test_score(model, testloader):
    model.eval()  ##test mode
    total = 0 ##total_num
    result_batch = [] ## all label 1's score
    correct = 0 ##correct_num
    with torch.no_grad():   ##eval不計算gradient 
        for data in testloader:
            tokens_tensors ,masks_tensors,labels = [t.to(device) for t in data]
            outputs = model(input_ids = tokens_tensors,
                           attention_mask = masks_tensors,
                           labels = labels)
            score = outputs[1]
            for idx in range(score.shape[0]):
                result_batch.append(score[idx][1])
#             result_score = 0
#             for num in score:
#                 for result in num:
#                     result_score += result
#             print(result_score)
    return result_batch

In [23]:
## train
train_input_dict = tokenizer(train_q_set, train_c_set, 
                             padding=True, 
                             truncation=True, 
                             return_tensors="pt",
                             max_length = 512)
TRAIN_BATCH_SIZE = 4
trainset = TrainDataset(train_input_dict, train_y_set) ##trainset參數如init
trainloader = DataLoader(trainset, batch_size = TRAIN_BATCH_SIZE, shuffle = True)  




In [21]:
## test 
test_input_dict = tokenizer(test_q_set, test_c_set, 
                             padding=True, 
                             truncation=True, 
                             return_tensors="pt",
                             max_length = 512)
TEST_BATCH_SIZE = 64
testset = TrainDataset(test_input_dict, test_y_set) ##trainset參數如init
testloader = DataLoader(testset, batch_size = TEST_BATCH_SIZE, shuffle = True)  

In [ ]:
# e = 0.1 ##label smooth的參數
# smooth_loss_fnc = F.CrossEntropyLoss()
for epoch in range(EPOCHS):
    i = 0
    running_loss = 0.0
    correct = 0
    total = 0

    for (i,data) in enumerate(trainloader):
        
        tokens_tensors ,   masks_tensors , labels  = [t.to(device) for t in data]
        outputs = model(input_ids=tokens_tensors, 
                             attention_mask=masks_tensors,
                             labels = labels)
        
        optimizer.zero_grad()
        logits = outputs[1]
#         max_idx = torch.argmax(logits,dim = -1)
#         for count in range(labels.size()[0]):
#             for idx in range(len(logits[count])):
#                 if(idx == max_idx[count]):
#                     logits[count][idx] *= 1/(1-e)
#                 else:
#                     logits[count][idx] -= e/(num_labels-1)
#         for count in range(labels.size()[0]):
#             print(count)
#             for idx in range(len(logits[count])):
#                 print(idx)
#                 logits[count][idx] += e/(num_labels-1)
#         smooth_loss = smooth_loss_fnc(logits,labels)
        pred = torch.argmax(logits,dim = -1)
        loss = outputs[0]
#         loss += smooth_loss
#         loss = bert_outputs[0]*0.5 +weight_loss*0.5
        
        loss.backward()
        optimizer.step()
        total += pred.size()[0]
        correct += (pred == labels).sum().item()
        running_loss += loss.item()
        
        print(f'\rEpoch [{epoch+1}/{EPOCHS}] {i}/{len(trainloader)} Loss: {running_loss:.4f} Acc : {(correct/total):.4f}', end='')
    test_acc = get_test_acc(model,testloader)
    ##all save
    torch.save(model.state_dict(),'./HW6_model/HW6_roberta_baseline_' + str(epoch) + '.pkl')
    print('   test acc:' , test_acc)
    with open('./hw6_roberta_test_score'+str(epoch)+'.txt','w') as f:
        f.write(str(test_acc))
        f.write('\n')
        f.close()

Epoch [1/3] 7845/7846 Loss: 4456.6424 Acc : 0.7499   test acc: 0.75
Epoch [2/3] 3036/7846 Loss: 1725.7231 Acc : 0.7466

In [ ]:
test_q = pd.read_csv('./test_queries.csv')
test_query_list = test_q['query_id'].tolist()
test_query_data = [t.lower() for t in test_q['query_text']]
rerank_idx_list = [t.split() for t in test_q['bm25_top1000']]
rerank_score_list = [t.split() for t in test_q['bm25_top1000_scores']]
result_score_list = []
print(test_query_data[0])

In [ ]:
result_score_list = np.zeros([80,1000]) ##分數
for i in range(len(rerank_score_list)):
    for j in range(len(rerank_score_list[0])):
        result_score_list[i][j] = float(rerank_score_list[i][j])
after_add_list = np.zeros([80,1000])

In [ ]:
all_ans_list = []
for i in range(80):
    print(i)
    ans_list = []
    idx_list = rerank_idx_list[i]
    score_list = result_score_list[i].copy()
    input_q = []
    input_c = []
    no_content_list = []
    for j in range(len(idx_list)):
        find = doc_dict[idx_list[j]] ## idx 
        if type(doc_text[find]) != float:
            text = clean_string(doc_text[find])
            input_q.append(test_query_data[i])
            input_c.append(text)
        else:
            no_content_list.append(j) ##空的
    batch_test_input_dict = tokenizer(input_q, input_c, 
                            padding=True, 
                            truncation=True, 
                            return_tensors="pt",
                            max_length = 512)
    TEST_BATCH_SIZE = 64
    batch_testset = TrainDataset(batch_test_input_dict,torch.zeros([len(idx_list)], dtype=torch.long)) ##trainset參數如init 給label沒差
    batch_testloader = DataLoader(batch_testset, batch_size = TEST_BATCH_SIZE, shuffle = False)  
    score = get_test_score(model,batch_testloader)
    no_content_count = 0
    for k in range(len(score)):
#         print(k)
#         print(score[k])
        if k not in no_content_list:
            score_list[k+no_content_count] +=  7 * score[k]
        else:
            no_content_count += 1
    after_add_list[i] = score_list
    ans_idx_list = sorted(range(1000),reverse = True,key = lambda k : score_list[k])
    for j in range(1000):
        ans_list.append(rerank_idx_list[i][ans_idx_list[j]])
    all_ans_list.append(ans_list)

In [ ]:
with open('./hw6_result_BM25_roberta_baseline.txt','w') as f:
    f.write('query_id,ranked_doc_ids\n')
    for i in range(80): 
        f.write(str(test_query_list[i]))
        f.write(',')
        for j in range(len(all_ans_list[0])):
            f.write(all_ans_list[i][j])
            f.write(' ')
        f.write('\n')
    f.close()